In [1]:
import pandas as pd
import numpy as np

In [2]:
imdb1 = pd.read_csv("zippedData/imdb.name.basics.csv.gz")
imdb2 = pd.read_csv("zippedData/imdb.title.akas.csv.gz")
imdb3 = pd.read_csv("zippedData/imdb.title.basics.csv.gz")
imdb4 = pd.read_csv("zippedData/imdb.title.crew.csv.gz")
imdb5 = pd.read_csv("zippedData/imdb.title.principals.csv.gz")
imdb6 = pd.read_csv("zippedData/imdb.title.ratings.csv.gz")

# IMDB Web Scraped Data

In [3]:
# cleaning primary keys in imdb dfs
imdb2['title_id'] = imdb2['title_id'].str.strip()
imdb3['tconst'] = imdb3['tconst'].str.strip()
imdb4['tconst'] = imdb4['tconst'].str.strip()
imdb5['tconst'] = imdb5['tconst'].str.strip()
imdb6['tconst'] = imdb6['tconst'].str.strip()

In [4]:
# dropping duplicates
imdb2.drop_duplicates(subset=['title_id'], inplace=True)

In [5]:
imdb3.drop_duplicates(subset=['tconst'], inplace=True)
imdb4.drop_duplicates(subset=['tconst'], inplace=True)
imdb5.drop_duplicates(subset=['tconst'], inplace=True)
imdb6.drop_duplicates(subset=['tconst'], inplace=True)

In [6]:
# importing web scrapes
web_scrape1 = pd.read_csv('imdb_monetary_data_16590.csv')
web_scrape1.set_index(['title_id'], inplace=True)

In [7]:
web_scrape2 = pd.read_csv('imdb_monetary_data_16591_to_25214_.csv')
web_scrape2.set_index(['title_id'], inplace=True)

In [8]:
web_scrape3 = pd.read_csv('imdb_monetary_data_29214_.csv')
web_scrape3.set_index(['title_id'], inplace=True)

In [9]:
web_scrape4 = pd.read_csv('imdb_monetary_data_33736_.csv')
web_scrape4.set_index(['title_id'], inplace=True)

In [10]:
web_scrape5 = pd.read_csv('imdb_monetary_data_49857_.csv')
web_scrape5.set_index(['title_id'], inplace=True)

In [11]:
# concatenating webscraped data
web_scrape = pd.concat([web_scrape1, web_scrape2, web_scrape3, web_scrape4, web_scrape5])
print(web_scrape.columns, end='\n\n')
print(web_scrape.shape, end='\n\n')
print(web_scrape.isna().sum())

Index(['Unnamed: 0', 'Country', 'Budget', 'Opening Weekend USA', 'Gross USA',
       'Cumulative Worldwide Gross', 'Runtime'],
      dtype='object')

(49922, 7)

Unnamed: 0                        0
Country                          67
Budget                        34723
Opening Weekend USA           46599
Gross USA                     46418
Cumulative Worldwide Gross    40513
Runtime                       12174
dtype: int64


In [12]:
# dropping unnamed column
web_scrape.drop(['Unnamed: 0'], axis=1, inplace=True)
web_scrape

,Country,Budget,Opening Weekend USA,Gross USA,Cumulative Worldwide Gross,Runtime
title_id,,,,,,
tt0369610,USA,"$150,000,000","$208,806,270,","$652,270,625","$1,670,401,444",124 min
tt0381957,UK,NaN,NaN,NaN,NaN,NaN
tt0401729,USA,"$250,000,000","$30,180,188,","$73,078,100","$284,139,100",132 min
tt10010134,France,NaN,NaN,NaN,NaN,52 min
tt10027708,Australia,NaN,NaN,NaN,NaN,94 min
...,...,...,...,...,...,...
tt2707842,USA,NaN,NaN,NaN,NaN,NaN
tt2708686,Spain,NaN,NaN,NaN,NaN,NaN
tt2709034,USA,"$5,000",NaN,NaN,NaN,132 min


In [13]:
# cleaning title_id (index)
web_scrape.index = web_scrape.index.str.strip()

In [14]:
# merging imdb2
df = web_scrape.merge(imdb2, left_on=web_scrape.index, right_on=['title_id'])

In [15]:
df = df.merge(imdb3, left_on=['title_id'], right_on=['tconst'])

In [16]:
df = df.merge(imdb4, left_on=['title_id'], right_on=['tconst'])

In [17]:
df = df.merge(imdb5, left_on=['title_id'], right_on=['tconst'])

In [18]:
df = df.merge(imdb6, how='left', left_on=['title_id'], right_on=['tconst'])

In [19]:
df

,Country,Budget,Opening Weekend USA,Gross USA,Cumulative Worldwide Gross,Runtime,title_id,ordering_x,title,region,...,writers,tconst_x,ordering_y,nconst,category,job,characters,tconst_y,averagerating,numvotes
0,USA,"$150,000,000","$208,806,270,","$652,270,625","$1,670,401,444",124 min,tt0369610,10,Джурасик свят,BG,...,"nm0415425,nm0798646,nm1119880,nm2081046,nm0000341",tt0369610,10,nm0189777,producer,producer,NaN,tt0369610,7.0,539338.0
1,UK,NaN,NaN,NaN,NaN,NaN,tt0381957,1,Cloud Cuckoo Land,GB,...,"nm1458371,nm1461088",tt0381957,10,nm1108038,composer,NaN,NaN,tt0381957,7.6,23.0
2,USA,"$250,000,000","$30,180,188,","$73,078,100","$284,139,100",132 min,tt0401729,10,Джон Картер: Мiж двох свiтiв,UA,...,"nm0004056,nm0028764,nm0149290,nm0123194",tt0401729,10,nm0606640,producer,producer,NaN,tt0401729,6.6,241792.0
3,France,NaN,NaN,NaN,NaN,52 min,tt10010134,1,Versailles Rediscovered - The Sun King's Vanis...,NaN,...,"nm2205937,nm1355962,nm10542772,nm0873469",tt10010134,10,nm0674930,editor,NaN,NaN,tt10010134,7.0,5.0
4,Australia,NaN,NaN,NaN,NaN,94 min,tt10027708,1,Miguelito - Canto a Borinquen,NaN,...,nm3495624,tt10027708,10,nm7089722,editor,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49783,USA,NaN,NaN,NaN,NaN,NaN,tt2707842,1,Holed-Up,US,...,nm3761418,tt2707842,10,nm5521732,actress,NaN,"[""Grace Fief""]",tt2707842,3.2,15.0
49784,Spain,NaN,NaN,NaN,NaN,NaN,tt2708686,1,Prodigy-man,ES,...,nm4798652,tt2708686,1,nm5521937,actress,NaN,"[""Doctora Ribalta""]",NaN,NaN,NaN
49785,USA,"$5,000",NaN,NaN,NaN,132 min,tt2709034,1,Beyond Reason: The Science of Self Discovery,US,...,NaN,tt2709034,1,nm4286803,self,NaN,"[""Himself""]",NaN,NaN,NaN
49786,USA,NaN,NaN,NaN,NaN,63 min,tt2713440,1,The World Famous Kid Detective,US,...,nm5450673,tt2713440,10,nm5524745,actor,NaN,"[""Nina Kid""]",tt2713440,6.9,48.0


In [20]:
# dropping unnecessary columns
df.drop(['ordering_x', 'language', 'attributes', 'original_title', 'Runtime', 'is_original_title', 'tconst_x', 'tconst_y', 'types', 'category', 'job', 'characters', 'ordering_y', 'title'], axis=1, inplace=True)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 49788 entries, 0 to 49787
Data columns (total 16 columns):
Country                       49725 non-null object
Budget                        15183 non-null object
Opening Weekend USA           3322 non-null object
Gross USA                     3502 non-null object
Cumulative Worldwide Gross    9406 non-null object
title_id                      49788 non-null object
region                        44004 non-null object
primary_title                 49788 non-null object
start_year                    49788 non-null int64
runtime_minutes               43230 non-null float64
genres                        48946 non-null object
directors                     49112 non-null object
writers                       40236 non-null object
nconst                        49788 non-null object
averagerating                 32781 non-null float64
numvotes                      32781 non-null float64
dtypes: float64(3), int64(1), object(12)
memory usage: 6.5+ 

# Data Cleaning

In [22]:
# Dropping Null Values
df = df.dropna(subset=['Opening Weekend USA', 'Gross USA', 'Cumulative Worldwide Gross'], how='all')

In [23]:
# Cumulative Worldwide Gross replace NaN with 0
df['Cumulative Worldwide Gross'] = df['Cumulative Worldwide Gross'].fillna(0)

/Users/davidbruce/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [24]:
# Movies from India contain 'INR' substring and need to be converted to USD or dropped
INR = df.loc[df['Cumulative Worldwide Gross'].str.contains('INR') == True]
NPR = df.loc[df['Cumulative Worldwide Gross'].str.contains('NPR') == True]

In [25]:
df = df.drop(df.loc[df['Cumulative Worldwide Gross'].str.contains('INR') == True].index)

In [26]:
df = df.drop(df.loc[df['Cumulative Worldwide Gross'].str.contains('NPR') == True].index)

In [27]:
def clean_currency(x):
    if isinstance(x, str):
        x = x.replace('$', '').replace(',', '')
        return x
    return(x)

# df['Cumulative Worldwide Gross'] = df['Cumulative Worldwide Gross'].str.replace(',', '').str.replace('$', '')

In [28]:
df['Cumulative Worldwide Gross'] = df['Cumulative Worldwide Gross'].map(lambda x: clean_currency(x))

In [29]:
df['Cumulative Worldwide Gross'] = pd.to_numeric(df['Cumulative Worldwide Gross'])

In [30]:
df['Cumulative Worldwide Gross'][:5]

0     1670401444
2      284139100
8     1025467803
9        2704326
12       1424100
Name: Cumulative Worldwide Gross, dtype: int64

In [31]:
# Gross USA 
df['Gross USA'] = df['Gross USA'].fillna(0)

In [32]:
df['Gross USA'] = df['Gross USA'].map(lambda x: clean_currency(x))

In [33]:
df['Gross USA'] = pd.to_numeric(df['Gross USA'])

In [34]:
df['Gross USA'][:5]

0     652270625
2      73078100
8     334191110
9             0
12            0
Name: Gross USA, dtype: int64

In [35]:
# Opening Weekend USA
df['Opening Weekend USA'] = df['Opening Weekend USA'].fillna(0)

In [36]:
df['Opening Weekend USA'] = df['Opening Weekend USA'].map(lambda x: clean_currency(x))

In [37]:
df['Opening Weekend USA'] = pd.to_numeric(df['Opening Weekend USA'])

In [38]:
df['Opening Weekend USA'][:5]

0     208806270
2      30180188
8     116101023
9             0
12            0
Name: Opening Weekend USA, dtype: int64

In [46]:
df.shape

(9437, 16)

In [40]:
df.Budget

0        $150,000,000
2        $250,000,000
8        $200,000,000
9                 NaN
12                NaN
             ...     
49766             NaN
49768      $1,500,000
49774     $20,000,000
49775             NaN
49780             NaN
Name: Budget, Length: 9437, dtype: object

In [41]:
df['Budget'] = df['Budget'].map(lambda x: clean_currency(x))

In [105]:
df['Budget'] = df['Budget'].fillna('0')

In [127]:
def find_currencies(x):
    currencies = []
    for i in x:
        if i[:3].isalpha():
            currencies.append(i[:3])
    return np.unique(currencies)

In [128]:
budget_curr = df['Budget'].tolist()

In [132]:
list_of_curr = find_currencies(budget_curr).tolist()

In [134]:
list_of_curr

['ARS',
 'AUD',
 'BGL',
 'BRL',
 'CAD',
 'CHF',
 'CLP',
 'CNY',
 'COP',
 'CZK',
 'DKK',
 'EEK',
 'EGP',
 'EUR',
 'GBP',
 'HKD',
 'HUF',
 'IDR',
 'INR',
 'ISK',
 'ITL',
 'JPY',
 'KRW',
 'LTL',
 'LVL',
 'MXN',
 'MYR',
 'NGN',
 'NOK',
 'NZD',
 'PEN',
 'PHP',
 'PKR',
 'PLN',
 'RUR',
 'SEK',
 'SGD',
 'THB',
 'TRL',
 'TWD',
 'UAH',
 'VEB',
 'ZAR']

In [45]:
def conversion_rates(x):
    #if x[:3] == 'EUR':
        # left strip x and multiply by conversion rate for EUR
    # if x[:3] == 'INR':
        # left strip x and multiply by conversion rate for INR
    pass

In [115]:
df['Budget'].head(50)

0           150000000
2           250000000
8           200000000
9                   0
12                  0
16           52000000
20          125000000
22           27000000
24            5000000
27                  0
29           35000000
37                  0
39                  0
41           25000000
50                  0
51           20000000
52              60000
54      INR1900000000
55           45000000
58                  0
59             130000
63         EUR2870000
65            4500000
67          105000000
68           50000000
72                  0
76         EUR5500000
77           30000000
78          EUR200000
79          135000000
81                  0
86                  0
88                  0
91                  0
94                  0
96         GBP4000000
99                  0
105                 0
107     JPY1000000000
108                 0
109          20000000
110                 0
113                 0
119                 0
130                 0
132       